In [2]:
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

n = 150 # number of key cells
gamma = 1/2 # ratio between the length of green domain and red domain

seed = 10000
dataset='HOG'
proportions = [0.2, 0.4, 0.6, 0.8, 1.0]

for proportion in proportions:
    loaded_results = np.load(f"../../../../datasets/HOG/watermarked/{dataset}-{10000}.npy", allow_pickle=True).item()
    # watermarked_data = loaded_results['watermarked_data']
    watermarked_data = pd.read_csv(f"../../../../datasets/HOG/alteration/HOG-10000-{proportion}-0.csv")
    divide_seeds = loaded_results['divide_seeds']
    indices = loaded_results['indices'] 
    #添加水印
    digit_types = watermarked_data['target'].unique()
    digit_types.sort()
    green_cell = 0
    for idx, divide_seed in zip(indices, divide_seeds):
        np.random.seed(divide_seed)
        candidate_set = digit_types
        # 打乱cover_types的顺序
        shuffled_cover_types = list(digit_types)
        # print(shuffled_cover_types)
        np.random.shuffle(shuffled_cover_types)

        # 确保cover_types能被划分为两个相等大小的部分
        half_size = len(shuffled_cover_types) // 2

        # 划分成green_domain和red_domain
        green_domain = shuffled_cover_types[:half_size]
        red_domain = shuffled_cover_types[half_size:]

        if watermarked_data.loc[idx, 'target'] in green_domain:
            green_cell += 1
        
    z_score = (green_cell - n/2) / math.sqrt(n/4)

    print(f"{proportion}: The average z-score is ",z_score)


0.2: The average z-score is  7.021870595978444
0.4: The average z-score is  9.308061022576076
0.6: The average z-score is  5.388877434122992
0.8: The average z-score is  2.939387691339814
1.0: The average z-score is  1.1430952132988164


In [6]:
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

n = 50 # number of key cells
gamma = 1/2 # ratio between the length of green domain and red domain
p = 25
k = 500

seed = 10000
dataset='housing'

original_file = '../../../../datasets/boston_housing_prices/HousingData.csv'
origin = pd.read_csv(original_file)
proportions = [0.2, 0.4, 0.6, 0.8, 1.0]

for proportion in proportions:
    loaded_results = np.load(f"../../../../datasets/boston_housing_prices/watermarked/{dataset}-{10000}.npy", allow_pickle=True).item()
    # watermarked_data = loaded_results['watermarked_data']
    watermarked_data = pd.read_csv(f"../../../../datasets/boston_housing_prices/alteration/housing-10000-{proportion}-0.csv")
    divide_seeds = loaded_results['divide_seeds']
    indices = loaded_results['indices']
    green_cell = 0
    for idx, divide_seed in zip(indices, divide_seeds):
        np.random.seed(divide_seed)
        # 生成等分点
        intervals = np.linspace(-p, p, k + 1)
        # 将 [-p, p] 等分为 k 份
        segments = [(intervals[i], intervals[i + 1]) for i in range(k)]
        np.random.shuffle(segments)
        # 将 segments 分为 green domains 和 red domains
        half_k = k // 2
        green_domains = segments[:half_k]
        red_domains = segments[half_k:]

        difference = watermarked_data.loc[idx, 'MEDV'] - origin.loc[idx, 'MEDV']
        for low, high in green_domains:
            if low <= difference < high:
                green_cell += 1
                break
        
    z_score = (green_cell - n/2) / math.sqrt(n/4)

    print(f"{proportion}: The average z-score is ", z_score)







0.2: The average z-score is  5.65685424949238
0.4: The average z-score is  2.82842712474619
0.6: The average z-score is  1.697056274847714
0.8: The average z-score is  -0.565685424949238
1.0: The average z-score is  -3.676955262170047
